## Create a generative AI runbook to resolve security findings

## Module 3

```
// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0
```

### Introduction

In this module, you will practice what you learned from Module1 to complete the following
- Get a finding from Security Hub
- Use Claude 3 to summarize the finding result
- Use Claude 3 to generate Python code used for remediating the finding
- Verify the finding is resolved.

**NOTE**: You can use SHIFT + ENTER to run each of the cells in this workbook.

**In this example, you will get a failed finding from Security Hub for `GuardDuty should be enabled [GuardDuty.1]`**

#### Step 1: Get a finding from Security Hub

In this step, you will get a failed finding from Security Hub. In this example, we use the AWS SDK for Python (boto3) to access the Security Hub API.

The Python function will return a JSON representation of the finding. You can also view this finding by visiting Security Hub in the AWS console.

You can safely ignore `Note: you may need to restart the kernel to use updated packages.`

In [ ]:
# Python code from Module 1. Update to seach for `GuardDuty should be enabled [GuardDuty.1]`
%pip install -qU pip boto3 awscli botocore

import boto3

sh = boto3.client('securityhub')

def get_finding(finding_id):
    response = sh.get_findings(
        Filters={
                'ComplianceSecurityControlId': [
                    {
                        'Value': finding_id,
                        'Comparison': 'EQUALS'
                    }
                ],
                'ComplianceStatus':[
                    {
                        'Value': 'FAILED',
                        'Comparison': 'EQUALS'
                    }
                ]
            },
        MaxResults=1
    
    )
    return response['Findings']

######
### Update get_finding("XXX") with GuardDuty.1
######

sg_finding = get_finding("XXX")
print(sg_finding)

#### Step 2: Summarize the finding using Claude 3

In this step, you will use the Anthropic SDK, to access Claude 3 through Amazon Bedrock.

In [ ]:
import boto3

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

bedrock_client = boto3.client(service_name='bedrock-runtime')
inference_config = {"temperature": 0}

def get_completion(prompt, system_prompt=None):

    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
    }

    # Check if system_prompt is provided and add the system parameter to the converse_api_params dictionary
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]

    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text']

After loading the function, you can create your first prompt.

In [ ]:
# For the system variable, enter a prompt to set the context"
# For the prompt variable, update to ask Claude 3 to summarize the finding.


system = "You are a..."
prompt = f"""
Review the finding and ...

<finding>
{sg_finding}
</finding>

"""

print(get_completion(prompt,system))

#### Step 3 - Use Claude 3 to generate Python code used for finding discovery

Based on the output in step 2, Claude 3 recommends enabling Amazon GuardDuty

In this step, we will ask Claude 3 to generate remediation function to enable GuardDuty

In [ ]:
# Update the prompt to ask Claude 3 to generate a function to enable GuardDuty

prompt = f"""Review the finding.

<finding>
{sg_finding}
</finding>

Create a Python function to...

"""

print(get_completion(prompt, system_prompt=system))

Carefully review the output function. When you understand how it works, paste it into the next cell and run it. Depending on the prompt used, you may need to call the function as well. 

For example, add the function name and the Region to the bottom.

`enable_guardduty('us-west-2')`

In [ ]:
import boto3

#....

enable_guardduty('us-west-2')

### Step 4: Verify GuardDuty is enabled.

To verify if GuardDuty is enabled, navigate to the console.
https://console.aws.amazon.com/guardduty/